In [1]:
import numpy as np
from gaussians import Gaussians
import json
import cv2
import pickle
import torch
from torch import nn

In [2]:
def calc_dists(preds, target):
	#preds  =  preds.astype(np.float32)
	#target = target.astype(np.float32)
	#dists  = np.zeros(preds.shape[0], preds.shape[1]))
  #dists = np.linalg.norm(preds - target)
  dists = torch.linalg.norm(preds - target)
  #for n in range(preds.shape[0]):
    #for c in range(preds.shape[1]):
      #if target[n, c, 0] > 1 and target[n, c, 1] > 1:
        #normed_preds   =  preds[n, c, :] / normalize[n]
        #normed_targets = target[n, c, :] / normalize[n]
        #dists[c, n]    = np.linalg.norm(preds - target)
      #else:
        #dists[c, n]    = -1
  return dists

In [3]:
def dist_acc(dists, threshold = 0.5):
	dist_cal     = np.not_equal(dists, -1)
	num_dist_cal = dist_cal.sum()

	if num_dist_cal > 0:
		return np.less(dists[dist_cal], threshold).sum() * 1.0 / num_dist_cal
	else:
		return -1

In [4]:
with open('annotations/train.json') as f:
	data = json.load(f)

for i in range(len(data)):
    if (data[i]['image'] == '000004812.jpg'):
        img_name = data[i]['image']
        #img = data[i]['image']
        img = cv2.imread(img_name)
        print(type(img))
        kpt = np.asarray(data[i]['joints'], dtype=np.int32)
        if img.shape[0] != 368 or img.shape[1] != 368:
            kpt[:, 0] = kpt[:, 0] * (368 / img.shape[1])
            kpt[:, 1] = kpt[:, 1] * (368 / img.shape[0])
            img = cv2.resize(img, (368, 368))
            img = np.array(img)
        #print(kpt[0])
        #print(kpt.shape)
        gaussian = Gaussians()
        g_list = []
        g_list.append(kpt)
        filename = 'epoch0_pickle'
        device = "cuda"
        heatmap = gaussian.expected_to_gaussian(g_list).to(device)
        #print(heatmap.sum())
        infile = open(filename,'rb')
        model = pickle.load(infile)
        device = "cuda"
        img = torch.Tensor(img).unsqueeze(0).permute(0, 3, 1, 2).to(device)
        print(model(img).shape)
        output = model(img)
        softmax = nn.Softmax(dim=2)
        # get Softmax over 2D image channels
        input_view = output.view(output.shape[0], output.shape[1], -1)
        output_view = softmax(input_view)
        output = output_view.view(output.shape)
        infile.close()
        for j in range(16):
          if(not (kpt[j, 0] < 0 or kpt[j, 1] < 0)):
            dist = calc_dists(output[0, j], heatmap[0, j])
            print(output[0, j].sum())
            print(heatmap[0, j].sum())
            print(dist)

<class 'numpy.ndarray'>
torch.Size([1, 16, 46, 46])
tensor(1., device='cuda:0', grad_fn=<SumBackward0>)
tensor(1., device='cuda:0', dtype=torch.float16)
tensor(0.3446, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1., device='cuda:0', dtype=torch.float16)
tensor(0.3430, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1., device='cuda:0', dtype=torch.float16)
tensor(0.2992, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1., device='cuda:0', dtype=torch.float16)
tensor(0.3056, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1., device='cuda:0', dtype=torch.float16)
tensor(0.3424, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)
tensor(1., device='cuda:0', dtype=torch.float16)
tensor(0.3441, device='cuda:0', grad_